<a href="https://colab.research.google.com/github/sakshivohra/Image-Classification/blob/main/Skin_Diseases_Detect.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Convolution2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten

In [ ]:
from google.colab import files
uploaded = files.upload()

In [ ]:
model = Sequential()

Adding Convolutional layers

In [ ]:
model.add(Convolution2D(32,(3,3),input_shape = (64,64,3), activation = 'relu'))

In [ ]:
model.add(Convolution2D(32,(3,3),input_shape = (64,64,3), activation = 'relu'))

Adding Maxpooling layer

In [ ]:
model.add(MaxPooling2D(pool_size = (2,2)))

Add flattening layer

In [ ]:
model.add(Flatten())

In [ ]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_2 (Conv2D)            (None, 62, 62, 32)        896       
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 60, 60, 32)        9248      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 30, 30, 32)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 28800)             0         
Total params: 10,144
Trainable params: 10,144
Non-trainable params: 0
_________________________________________________________________


Adding hidden layers

In [ ]:
model.add(Dense(units=  128, kernel_initializer= 'uniform', activation = 'relu'))

In [ ]:
model.add(Dense(units=  128, kernel_initializer= 'uniform', activation = 'relu'))

In [ ]:
model.add(Dense(units= 64, kernel_initializer= 'uniform', activation = 'relu'))

In [ ]:
model.add(Dense(units= 64, kernel_initializer= 'uniform', activation = 'relu'))

Adding Output layer

In [ ]:
model.add(Dense(activation = 'softmax', units= 5, kernel_initializer= 'uniform'))

Preprocessing the data

In [ ]:
from keras.preprocessing.image import ImageDataGenerator

In [ ]:
train_datagen = ImageDataGenerator(rescale = 1./255, shear_range = 0.2, zoom_range = 0.2, height_shift_range = 0.2, width_shift_range = 0.2, horizontal_flip = True, vertical_flip = True)
test_datagen = ImageDataGenerator(rescale = 1)

Loading the train and test data

In [ ]:
!unzip Skin\ Diseases.zip -d Skin\ Diseases

Archive:  Skin Diseases.zip
   creating: Skin Diseases/Skin Diseases/
   creating: Skin Diseases/Skin Diseases/Skin Diseases/
   creating: Skin Diseases/Skin Diseases/Skin Diseases/test/
   creating: Skin Diseases/Skin Diseases/Skin Diseases/test/Acne/
  inflating: Skin Diseases/Skin Diseases/Skin Diseases/test/Acne/2Q__ (1).jpg  
  inflating: Skin Diseases/Skin Diseases/Skin Diseases/test/Acne/2Q__ (10).jpg  
  inflating: Skin Diseases/Skin Diseases/Skin Diseases/test/Acne/2Q__ (11).jpg  
  inflating: Skin Diseases/Skin Diseases/Skin Diseases/test/Acne/2Q__ (12).jpg  
  inflating: Skin Diseases/Skin Diseases/Skin Diseases/test/Acne/2Q__ (13).jpg  
  inflating: Skin Diseases/Skin Diseases/Skin Diseases/test/Acne/2Q__ (14).jpg  
  inflating: Skin Diseases/Skin Diseases/Skin Diseases/test/Acne/2Q__ (15).jpg  
  inflating: Skin Diseases/Skin Diseases/Skin Diseases/test/Acne/2Q__ (16).jpg  
  inflating: Skin Diseases/Skin Diseases/Skin Diseases/test/Acne/2Q__ (17).jpg  
  inflating: Skin D

In [ ]:
x_train = train_datagen.flow_from_directory(r'/content/Skin Diseases/Skin Diseases/Skin Diseases/train', target_size = (64, 64), batch_size = 32, class_mode = 'categorical')
x_test = test_datagen.flow_from_directory(r'/content/Skin Diseases/Skin Diseases/Skin Diseases/test', target_size = (64, 64), batch_size = 32, class_mode = 'categorical')

Found 2205 images belonging to 5 classes.
Found 550 images belonging to 5 classes.


In [ ]:
x_train.class_indices

{'Acne': 0, 'Melanoma': 1, 'Psoriasis': 2, 'Rosacea': 3, 'Vitiligo': 4}

In [ ]:
model.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])

Fitting the data

In [ ]:
model.fit_generator(x_train, steps_per_epoch =100, epochs = 150, validation_data = x_test, validation_steps = 63)

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/150
100/100 [==============================] - 28s 272ms/step - loss: 1.6057 - accuracy: 0.2193 - val_loss: 8.9518 - val_accuracy: 0.2436


Saving the model

In [ ]:
model.save("Skin_Diseases.h5")

In [ ]:
!unzip templates.zip

Archive:  templates.zip
  inflating: templates/base.html     
  inflating: templates/index.html    


In [ ]:
from __future__ import division, print_function
# coding=utf-8
import sys
import os
!pip install gevent
import gevent
import glob
import numpy as np
from keras.preprocessing import image
from keras.applications.imagenet_utils import preprocess_input, decode_predictions
from keras.models import load_model

from tensorflow.python.keras import backend
import tensorflow as tf
global graph

from skimage.transform import resize
# Flask utils
!pip install flask-ngrok
from flask_ngrok import run_with_ngrok
from flask import Flask, redirect, url_for, request, render_template
from werkzeug.utils import secure_filename
from gevent.pywsgi import WSGIServer

# Define a flask app
app = Flask(__name__)
run_with_ngrok(app)
MODEL_PATH = 'Skin_Diseases.h5'

# Load your trained model
model = load_model(MODEL_PATH)
       
@app.route('/', methods=['GET'])
def index():
    # Main page
    return render_template('index.html')


@app.route('/predict', methods=['GET', 'POST'])
def upload():
    if request.method == 'POST':
        # Get the file from post request
        f = request.files['file']

        # Save the file to ./uploads
        basepath = os.path.dirname(__file__)
        file_path = os.path.join(
            basepath, 'uploads', secure_filename(f.filename))
        f.save(file_path)
        img = image.load_img(file_path, target_size=(64, 64))
        x = image.img_to_array(img)
        x = np.expand_dims(x, axis=0)
        
        with graph.as_default():
            preds = model.predict_classes(x)
        index = ['Acne', 'Melanoma', 'Psoriasis', 'Rosacea', 'Vitiligo']
        text = "Prediction : "+index[preds[0]]

        return text
    
if __name__ == '__main__':
  
    app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off
 * Running on http://974e361ddc8f.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


127.0.0.1 - - [19/Mar/2021 07:51:45] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [19/Mar/2021 07:51:45] "GET /static/css/main.css HTTP/1.1" 404 -
127.0.0.1 - - [19/Mar/2021 07:51:45] "GET /static/js/main.js HTTP/1.1" 404 -
127.0.0.1 - - [19/Mar/2021 07:51:47] "GET /static/js/main.js HTTP/1.1" 404 -
127.0.0.1 - - [19/Mar/2021 07:51:47] "GET /static/css/main.css HTTP/1.1" 404 -
127.0.0.1 - - [19/Mar/2021 07:51:47] "GET /bgimg.jpg HTTP/1.1" 404 -
127.0.0.1 - - [19/Mar/2021 07:51:47] "GET /favicon.ico HTTP/1.1" 404 -
